In [2]:
!python --version

Python 3.10.9


In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import pickle 
import json 

# Step 1: Problem Statement 

# Step 2: Data Gathering 

In [4]:
df = pd.read_csv('medical_insurance.csv')
df

,age,gender,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


# EDA

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   gender    1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [9]:
df['gender'].value_counts().to_dict()
df['gender'].replace({'male': 1, 'female': 0}, inplace = True)

In [11]:
df['smoker'].value_counts().to_dict()
df['smoker'].replace({'no': 0, 'yes': 1}, inplace = True)

In [13]:
df.region.value_counts()  ## nominal >> one hot encoding 

southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64

In [19]:
df = pd.get_dummies(df, columns=['region'])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   int64  
 1   gender            1338 non-null   int64  
 2   bmi               1338 non-null   float64
 3   children          1338 non-null   int64  
 4   smoker            1338 non-null   int64  
 5   charges           1338 non-null   float64
 6   region_northeast  1338 non-null   uint8  
 7   region_northwest  1338 non-null   uint8  
 8   region_southeast  1338 non-null   uint8  
 9   region_southwest  1338 non-null   uint8  
dtypes: float64(2), int64(4), uint8(4)
memory usage: 68.1 KB


# Model Training

In [22]:
x = df.drop('charges', axis=1)
y = df['charges']

x_train,x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=17)
linear_model = LinearRegression() # GDA
linear_model.fit(x_train,y_train)

LinearRegression()

# Model Evaluation 

In [25]:
train_score = linear_model.score(x_train,y_train)
print(train_score)

test_score = linear_model.score(x_test,y_test)
print(test_score)

0.7519020180281468
0.7345433061333784


In [ ]:
# Bias >> high 
# Variance >> low 

In [26]:
knn_model = KNeighborsRegressor()
knn_model.fit(x_train,y_train)  ## lazy learner

KNeighborsRegressor()

In [27]:
train_score = knn_model.score(x_train,y_train)
print(train_score)

test_score = knn_model.score(x_test,y_test)
print(test_score)

0.42082538809056236
0.13806523817402327


In [31]:
scaler = MinMaxScaler()
arr = scaler.fit_transform(x)
x_scale = pd.DataFrame(arr, columns = x.columns)
x_scale

,age,gender,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,0.021739,0.0,0.321227,0.0,1.0,0.0,0.0,0.0,1.0
1,0.000000,1.0,0.479150,0.2,0.0,0.0,0.0,1.0,0.0
2,0.217391,1.0,0.458434,0.6,0.0,0.0,0.0,1.0,0.0
3,0.326087,1.0,0.181464,0.0,0.0,0.0,1.0,0.0,0.0
4,0.304348,1.0,0.347592,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1333,0.695652,1.0,0.403820,0.6,0.0,0.0,1.0,0.0,0.0
1334,0.000000,0.0,0.429379,0.0,0.0,1.0,0.0,0.0,0.0
1335,0.000000,0.0,0.562012,0.0,0.0,0.0,0.0,1.0,0.0
1336,0.065217,0.0,0.264730,0.0,0.0,0.0,0.0,0.0,1.0


In [32]:
x_scale_train,x_scale_test, y_train, y_test = train_test_split(x_scale,y, test_size=0.2, random_state=17)

In [34]:
knn_model = KNeighborsRegressor()
knn_model.fit(x_scale_train,y_train)  ## lazy learner

train_score = knn_model.score(x_scale_train,y_train)
print(train_score)

test_score = knn_model.score(x_scale_test,y_test)
print(test_score)

0.8493198307380913
0.754302385835028


In [ ]:
## Bias >> high 
## Variance >> high

## Save Model File 

In [38]:
with open('model.pkl','wb') as file: 
    pickle.dump(knn_model,file)

In [40]:
x.columns.tolist()

['age',
 'gender',
 'bmi',
 'children',
 'smoker',
 'region_northeast',
 'region_northwest',
 'region_southeast',
 'region_southwest']

In [41]:
project_data = {"column_names":x.columns.tolist(),
               "gender":{'male': 1, 'female': 0},
               "smoker":{"no":0,"yes":1}
               }

with open('project_data.json','w') as file:
    json.dump(project_data,file)

In [47]:
x_train

,age,gender,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
588,61,0,35.910,0,0,1,0,0,0
421,61,1,35.860,0,1,0,0,1,0
90,21,0,39.490,0,0,0,0,1,0
1147,20,0,31.920,0,0,0,1,0,0
934,32,1,37.180,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
752,64,1,37.905,0,0,0,1,0,0
406,33,0,24.310,0,0,0,0,1,0
143,29,1,29.735,2,0,0,1,0,0
1265,64,1,23.760,0,1,0,0,1,0


### USER DEFINED FUNCTION

In [66]:
age = 19
gender = "male"
bmi = 35.900
smoker = "yes"
region = "northwest"

# preprocessing 

if gender == "male":
    gender = 1
else: 
    gender = 0
    
if smoker == "no":
    smoker = 1
else: 
    smoker = 0

    
user_data = np.zeros(len(x.columns))
user_data[0] = age
user_data[1] = gender
user_data[2] = bmi
user_data[3] = smoker

search_region = 'region_'+region
index = np.where(x.columns == search_region)[0][0]
user_data[index] = 1

### Scaling the user data 
user_data_scale = scaler.transform([user_data])
print(user_data_scale)

result = knn_model.predict(user_data_scale)
result

[[0.02173913 1.         0.53645413 0.         0.         0.
  1.         0.         0.        ]]


C:\Users\Shridhar\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Shridhar\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


array([1833.63271])

In [51]:

x.columns

Index(['age', 'gender', 'bmi', 'children', 'smoker', 'region_northeast',
       'region_northwest', 'region_southeast', 'region_southwest'],
      dtype='object')

In [54]:
region = "southeast"
search_region = 'region_'+region
search_region

'region_southeast'

In [57]:
index = np.where(x.columns == search_region)[0][0]

7

In [14]:
df_v = pd.read_csv('medical_insurance.csv')
df_v

,age,gender,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [16]:
df_v['gender']  ## Categorcial >> nominal ### one hot encoding get_dummies 

0       female
1         male
2         male
3         male
4         male
         ...  
1333      male
1334    female
1335    female
1336    female
1337    female
Name: gender, Length: 1338, dtype: object

In [18]:
pd.get_dummies(df_v, columns=['gender'], drop_first= True)

,age,bmi,children,smoker,region,charges,gender_male
0,19,27.900,0,yes,southwest,16884.92400,0
1,18,33.770,1,no,southeast,1725.55230,1
2,28,33.000,3,no,southeast,4449.46200,1
3,33,22.705,0,no,northwest,21984.47061,1
4,32,28.880,0,no,northwest,3866.85520,1
...,...,...,...,...,...,...,...
1333,50,30.970,3,no,northwest,10600.54830,1
1334,18,31.920,0,no,northeast,2205.98080,0
1335,18,36.850,0,no,southeast,1629.83350,0
1336,21,25.800,0,no,southwest,2007.94500,0
